In [168]:
def find_main_connective(formula):
    # Remove outermost parentheses if they exist
    formula = formula.strip()
    negation = False
    if formula[0] == '¬':
        negation = True
        formula = formula[1:]
    if formula[0] == '(' and formula[-1] == ')':
        formula = formula[1:-1]
    # Count the number of open and close parentheses
    open_paren_count = 0
    main_connective = None
    main_connective_index = None
    for i, char in enumerate(formula):
        if char == '(':
            open_paren_count += 1
        elif char == ')':
            open_paren_count -= 1
        # Check for main connective if at the top level of parentheses
        elif open_paren_count == 0 and char in {'^', 'V'}:
            main_connective = char
            main_connective_index = i
            break  # Exit loop once main connective is found
    # If no main connective is found, assume it's a single literal
    if main_connective is None:
        return None, [formula], None
    else:
        # Find the index of the main connective where it occurs at the top level of parentheses
        for i, char in enumerate(formula):
            if negation == False:
                if i >= main_connective_index and char == main_connective:
                    return main_connective, [formula[:i-1], formula[i+2:]], negation
            elif negation == True:
                if i >= main_connective_index and char == main_connective:
                    return main_connective, ['¬' + formula[:i-1], '¬' + formula[i+2:]], negation 

def simplify_formula(formula):
    if len(formula)<=2:
        return formula
    else:
        if formula[0] == '¬' and formula[1] == '¬':
            counter = 0
            for i in formula:
                if i == '¬':
                    counter += 1
                elif i != '¬':
                    break    
            if counter % 2 == 0:
                return formula[counter:]
            else:
                return ['¬' + formula[counter:]]  
        else:
            return formula    
        
def open_formula_check(open_formulas, num_new):
    # print("Open Formula check")
    flags = []
    for i in range(1, num_new+1):
        # print("i: ",i)
        formula = open_formulas[-i]
        # print("formula for checking: ",formula)
        negated_formula = '¬' + formula
        negated_formula = simplify_formula(negated_formula)
        # print("negated formula: ",negated_formula)
        if negated_formula in open_formulas:
            open_formulas.remove(negated_formula)
            open_formulas.remove(formula)
            flags.append(formula)
    return open_formulas, flags

def resolve_2(formula, open_formulas, branch_count):
    print('Formula: ',formula)
    cleaned_formulas = []
    op, new_formulas, negation_flag = find_main_connective(formula)
    print(op, new_formulas, negation_flag)
    for formula in new_formulas:
        formula = simplify_formula(formula)
        print("simplified: ",formula)
        cleaned_formulas.append(formula)
        print("cleaned: ",cleaned_formulas)
        open_formulas.append(formula)
    #open formula check
    print("open formulas: ",open_formulas)
    # print(len(cleaned_formulas))
    open_formulas, to_close = open_formula_check(open_formulas, len(cleaned_formulas))
    branch_count -= len(to_close)
    #edit tree with to close
    if op == "V":
        if negation_flag == False:
            #branch
            branch_count += 1
        # elif negation_flag == True:
        #     #add both
    elif op == "^":
        if negation_flag == True:
            #BRANCH
            branch_count += 1
        # elif negation_flag == False:
        #     #add both
        #     continue
    for formula in cleaned_formulas:
        # print("enter")
        # print("cleaned 2: ",cleaned_formulas)
        op, a, b = find_main_connective(formula)
        # print(op, a, b)
        if op is not None:
            resolve_2(formula, open_formulas, branch_count)
    return branch_count, open_formulas

def resolve(kb):
    open_formulas = []
    branch_count = 1
    #add negated goal to tree
    open_formulas.append("¬" + kb[0])
    kb[0] = "¬" + kb[0]
    for formula in kb:
        resolve_2(formula, open_formulas, branch_count)
        print("Branch count: ",branch_count)
    open_formulas, to_close = open_formula_check(open_formulas, 1)
    branch_count -= len(to_close)
    if branch_count > 0:
        resolved = True
    elif branch_count == 0:
        resolved = False
    return resolved

In [169]:
formulas = ['T', '(P ^ Q)', '(¬P V R)','(¬(R ^ S) V T)', '(¬Q V S)'] # Alice Problem
# formulas = ['(¬(Q ^ (¬P V Q)) V P)']
# formulas = ['(¬(P ^ (¬P V Q)) V Q)']
formulas = ['¬R','(¬P V R)', '(¬R V S)', '(¬¬S V ¬Q)', '(¬P V R)', '¬T']
resolved = resolve(formulas)
if resolved == True:
    print("Model Exists, Goal is Unsatisfiable")
elif resolved == False:
    print("Model Does not Exist, Goal is a Tautology")

Formula:  ¬¬R
None ['¬R'] None
simplified:  ¬R
cleaned:  ['¬R']
open formulas:  ['¬¬R', '¬R']
Branch count:  1
Formula:  (¬P V R)
V ['¬P', 'R'] False
simplified:  ¬P
cleaned:  ['¬P']
simplified:  R
cleaned:  ['¬P', 'R']
open formulas:  ['¬¬R', '¬R', '¬P', 'R']
Branch count:  1
Formula:  (¬R V S)
V ['¬R', 'S'] False
simplified:  ¬R
cleaned:  ['¬R']
simplified:  S
cleaned:  ['¬R', 'S']
open formulas:  ['¬¬R', '¬P', '¬R', 'S']
Branch count:  1
Formula:  (¬¬S V ¬Q)
V ['¬¬S', '¬Q'] False
simplified:  S
cleaned:  ['S']
simplified:  ¬Q
cleaned:  ['S', '¬Q']
open formulas:  ['¬¬R', '¬P', '¬R', 'S', 'S', '¬Q']
Branch count:  1
Formula:  (¬P V R)
V ['¬P', 'R'] False
simplified:  ¬P
cleaned:  ['¬P']
simplified:  R
cleaned:  ['¬P', 'R']
open formulas:  ['¬¬R', '¬P', '¬R', 'S', 'S', '¬Q', '¬P', 'R']
Branch count:  1
Formula:  ¬T
None ['T'] None
simplified:  T
cleaned:  ['T']
open formulas:  ['¬¬R', '¬P', 'S', 'S', '¬Q', '¬P', 'T']
Branch count:  1
Model Exists, Goal is Unsatisfiable


<h1> Ignore Below </h1>

In [25]:
lit = str("(¬(P V R) V T)")
# lit.split('V')

In [26]:
def break_formula(formula, op):
    # This function breaks a formula into two subformulas based on the main connective.
    if op is None:
        return [formula]
    else:
        negation = False
        if formula[0] != '¬':
            temp = formula.split(op)
            broken = []
            for i in temp: # remove unresolved brackets if any
                if i[0] == '(' and i[-1] != ')':
                    j = i[1:]
                    broken.append(j.strip())
                elif i[-1] == ')' and i[0] != '(':
                    j = i[:-1]
                    broken.append(j.strip())
                else:
                    broken.append(i.strip())
        elif formula[0] == '¬':
            negation = True
            temp = formula[1:].split(op)
            temp = ['¬' + i for i in temp]
            broken = []
            for i in temp:
                if i[0] == '(' and i[-1] != ')':
                    j = i[1:]
                    broken.append(j.strip())
                elif i[-1] == ')' and i[0] != '(':
                    j = i[:-1]
                    broken.append(j.strip())
                else:
                    broken.append(i.strip())
    return broken, negation

break_formula(lit, 'V')

(['¬(P', 'R)', 'T'], False)

In [54]:
def simplify_formula(formula):
    if formula[0] == '¬' and formula[1] == '¬':
        counter = 0
        for i in formula:
            if i == '¬':
                counter += 1    
        if counter % 2 == 0:
            return formula[counter:]
        else:
            return ['¬' + formula[counter:]]
def open_formula_check(open_formulas, num_new):
    flags = []
    if num_new==1:
        formula = open_formulas[-num_new]
        negated_formula = '¬' + formula
        negated_formula = simplify_formula(negated_formula)
        if negated_formula in open_formulas:
            open_formulas.remove(negated_formula)
            open_formulas.remove(formula)
            flags.append(formula)
    else:
        for i in range(1, num_new):
            formula = open_formulas[-i]
            negated_formula = '¬' + formula
            negated_formula = simplify_formula(negated_formula)
            if negated_formula in open_formulas:
                open_formulas.remove(negated_formula)
                open_formulas.remove(formula)
                flags.append(formula)
    return open_formulas, flags

open_formula_check(['¬(P V R) V T', 'P V R','T', 'P','¬T'], 2)

(['¬(P V R) V T', 'P V R', 'P'], ['¬T'])

Formula:  ¬(¬(P ^ (¬P V Q)) V Q)
V ['¬¬(P ^ (¬P V Q))', '¬Q'] True
simplified:  (P ^ (¬P V Q))
cleaned:  ['(P ^ (¬P V Q))']
simplified:  ¬Q
cleaned:  ['(P ^ (¬P V Q))', '¬Q']
open formulas:  ['¬(¬(P ^ (¬P V Q)) V Q)', '(P ^ (¬P V Q))', '¬Q']
Formula:  (P ^ (¬P V Q))
^ ['P', '(¬P V Q)'] False
simplified:  P
cleaned:  ['P']
simplified:  (¬P V Q)
cleaned:  ['P', '(¬P V Q)']
open formulas:  ['¬(¬(P ^ (¬P V Q)) V Q)', '(P ^ (¬P V Q))', '¬Q', 'P', '(¬P V Q)']
Formula:  (¬P V Q)
V ['¬P', 'Q'] False
simplified:  ¬P
cleaned:  ['¬P']
simplified:  Q
cleaned:  ['¬P', 'Q']
open formulas:  ['¬(¬(P ^ (¬P V Q)) V Q)', '(P ^ (¬P V Q))', '¬Q', 'P', '(¬P V Q)', '¬P', 'Q']
Branch count:  1


(False, 'T')